In [9]:
using Pkg
Pkg.activate(".")

using JuMP
using MosekTools
using DynamicPolynomials
using MultivariatePolynomials
using TSSOS
using LinearAlgebra

# Parameters 
ϵ = 10^(-5)

@polyvar x0 w
# @polyvar w

function scale_list(list)
    # scale a list of floats so that the largest absolute value is 1
    a = abs(minimum(list))
    b = abs(maximum(list))
    if a <= b
        list = list/b
    else
        list = list/a
    end
end

function interpolation(deg)
    # synthesize Craig interpolation using the standard Putinar theorem
    # deg: degree of interpolation template, h(x,y)
    
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    h,hc,hb = add_poly!(model, xvars, deg)
    
    # just avoid bugs when yvars is empty
    if length(yvars)==0
        phivars = xvars
    else
        phivars = [xvars; yvars]
    end
    if length(zvars)==0
        psivars = xvars
    else
        psivars = [xvars; zvars]
    end

    # homogenization
    for i = 1:length(s1)
        model,info1 = add_psatz!(model, h-1,  phivars, s1[i], [],  d_relax, QUIET=true, CS=false, TS=false, Groebnerbasis=true)
    end
    
    for i = 1:length(s2)
        model,info2 = add_psatz!(model, -h-1, psivars, s2[i], [],  d_relax, QUIET=true, CS=false, TS=false,  Groebnerbasis=true)
    end
    for i in hc
        @constraint(model, -1<=i<=1)
    end
    # @constraint(model, sum(hc)==1)

    optimize!(model)
    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("termination status: $status")
        status = primal_status(model)
        println("solution status: $status")
    end
    hc = value.(hc)
    hc = scale_list(hc)   
    for i in 1:length(hc)
        if hc[i] <= ϵ && hc[i] >= -ϵ
            hc[i] = 0
        end
    end
    return hc'*hb
end

function interpolation_homo(deg)
    # synthesize Craig interpolation using the homogenization formulation
    # deg: degree of interpolation template, h(x,y)

    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    h,hc,hb = add_poly!(model, xvars, deg)
        
    if length(yvars)==0
        phivars = xvars
    else
        phivars = [xvars; yvars]
    end

    if length(zvars)==0
        psivars = xvars
    else
        psivars = [xvars; zvars]
    end

    for i = 1:length(s1)
        model,info = add_psatz!(model, homogenize(h, x0),  [x0; phivars], [homogenize.(s1[i], x0); x0], [1-sum([x0;phivars].^2)], d_relax, QUIET=true, CS=false, TS=false, Groebnerbasis=true)
    end    
    for i = 1:length(s2)
        model,info = add_psatz!(model, homogenize(-h, x0), [x0; psivars], [homogenize.(s2[i], x0); x0], [1-sum([x0;psivars].^2)], d_relax, QUIET=true, CS=false, TS=false, Groebnerbasis=true)
    end
    for i in hc
        @constraint(model, -1<=i<=1)
    end
    # @constraint(model, sum(hc)==1)

    optimize!(model)
    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("termination status: $status")
        status = primal_status(model)
        println("solution status: $status")
    end
    hc = value.(hc)  
    hc = scale_list(hc)    
    for i in 1:length(hc)
        if hc[i] <= ϵ && hc[i] >= -ϵ
            hc[i] = 0
        end
    end
    return hc'*hb
end

function interpolation_semi(deg)
    # synthesize semialgebraic interpolant
    # deg: degree of interpolation template, h1(x) h2(x)
    
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    h1,hc1,hb1 = add_poly!(model, xvars, deg)
    h2,hc2,hb2 = add_poly!(model, xvars, deg)
    
    if length(yvars)==0
        phivars = xvars
    else
        phivars = [xvars; yvars]
    end

    if length(zvars)==0
        psivars = xvars
    else
        psivars = [xvars; zvars]
    end
    
    for i = 1:length(s1)
        model,info = add_psatz!(model, homogenize(h1+w*h2, x0),  [x0; w; phivars], [homogenize.(s1[i], x0); x0; w], [1-sum([x0;phivars].^2)-w^2; sum([x0;xvars].^2)-w^2], d_relax, QUIET=true, CS=false, TS=false, Groebnerbasis=true)
    end    
    for i = 1:length(s2)
        model,info = add_psatz!(model, homogenize(-h1-w*h2, x0), [x0; w; psivars], [homogenize.(s2[i], x0); x0; w], [1-sum([x0;psivars].^2)-w^2; sum([x0;xvars].^2)-w^2], d_relax, QUIET=true, CS=false, TS=false, Groebnerbasis=true)
    end
    
    for i in hc1
        @constraint(model, -1<=i<=1)
    end
    for i in hc2
        @constraint(model, -1<=i<=1)
    end

    optimize!(model)
    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("termination status: $status")
        status = primal_status(model)
        println("solution status: $status")
    end
    hc1 = value.(hc1)      
    hc2 = value.(hc2)
    h_val = hc1'*hb1 + w*hc2'*hb2
    coef_list = coefficients(h_val)
    coef_list = scale_list(coef_list)
    for i in 1:length(coef_list)
        if coef_list[i] <= ϵ && coef_list[i] >= -ϵ
            coef_list[i] = 0
        end
    end
    h_val = dot(coef_list, monomials(h_val))
    return h_val
end




  Activating project at `~/Documents/code/Interpolation`


interpolation_semi (generic function with 1 method)

In [10]:
# compute interpolation using three methods
function run_tssos(name, method1, method2, method3)
    include("./Benchmarks/"*name*".jl");

    # print problem instance (so that Mathematica can read)
    file = open("./Results/problem/"*name*".txt", "w");
    write(file, "{")
    for i = 1:length(xvars) 
        if i<=length(xvars)-1
            write(file, string(xvars[i])*",")
        else
            write(file, string(xvars[i]))
        end
    end
    write(file, "}\n")
    write(file, "{")
    for i = 1:length(yvars) 
        if i<=length(yvars)-1
            write(file, string(yvars[i])*",")
        else
            write(file, string(yvars[i]))
        end
    end
    write(file, "}\n")
    write(file, "{")
    for i = 1:length(zvars) 
        if i<=length(zvars)-1
            write(file, string(zvars[i])*",")
        else
            write(file, string(zvars[i]))
        end
    end
    write(file, "}\n")
    for k = [s1, s2]
        write(file, "{")
        for i = 1:length(k)
            write(file, "{")
            for j = 1:length(k[i])
                write(file, Base.replace(string(k[i][j]),"e"=>"*10^"))
                if j < length(k[i])
                    write(file, ",") 
                end
            end
            write(file, "}")
            if i < length(k)
                write(file, ",")
            end
        end
        write(file, "}\n")
    end
    close(file)

    if method1 
        # print results based on the method in CAV20
        file = open("./Results/sufficient/"*name*".txt", "w");
        stats = @timed h = interpolation(deg)
        println("using CAV20 technique:")
        @show h
        write(file, Base.replace(string(h),"e"=>"*10^")*"\n")
        write(file, string(stats.time)*"\n") 
        close(file)
    end

    if method2
        # print homogenization approach results
        file = open("./Results/homo/"*name*".txt", "w");
        stats = @timed h = interpolation_homo(deg)
        println("Polynomial Interpolant:");
        @show h
        write(file, Base.replace(string(h),"e"=>"*10^")*"\n")
        write(file, string(stats.time)*"\n") 
        close(file)
    end
    
    if method3
        # print semialgebraic approach results
        file = open("./Results/nonpoly/"*name*".txt", "w");
        stats = @timed h = interpolation_semi(deg)
        println("Semialgebraic Interpolant:")
        @show h
        write(file, Base.replace(string(h),"e"=>"*10^")*"\n")
        write(file, string(stats.time)*"\n") 
        close(file)
    end
end

run_tssos (generic function with 1 method)

In [33]:
# methods
putinar = true
homogenization = true
semialgebraic = true

run_tssos("ex4", putinar, homogenization, semialgebraic)


termination status: INFEASIBLE
solution status: NO_SOLUTION
using CAV20 technique:
h = -0.000206843152785438x³ + 0.03136330813862681x²y + 0.013075005163987317xy² + 0.03302552685364803y³ + 0.10560230930263119x² + xy + 0.07966716045285478y² + 0.009312225437852302x - 0.008821406246921664y - 0.06004108898895729
Polynomial Interpolant:
h = -0.006903317812879289x³ + 0.020159661408869645x²y - 0.036542165234497076xy² + 0.016438460490184475y³ + 0.35883964577211874x² + xy + 0.31061331327037656y² + 0.04946084553402387x + 0.01889308625007449y - 0.35621385580764087


Semialgebraic Interpolant:
h = 0.2319983685266201wx² - 0.013332570273280156wxy + 0.20355733652455088wy² - 0.037034608636155646x³ + 0.051493129085329464x²y - 0.03844297393364402xy² + 0.038959500764794375y³ + 0.02407258064453592wx - 0.013051778695834803wy - 0.3225312558622174x² + xy - 0.2816439132018455y² + 0.056443176628256465w + 0.01446435742491452x - 0.0009318418609549722y - 0.0440220932808297
